In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, matplotlib as mpl
from scipy.special import logsumexp
%matplotlib inline

In [13]:
df = pd.read_csv('results/paper2_mock_1.csv', skiprows=1)

In [15]:
df.logwt.isna().any()

True

In [10]:
def merge_batches(df, nlive_init, nlive_batch):
    ntot = len(df)
    # Compute logvol for entire merged run
    dlv_init = -np.log((nlive_init + 1.) / nlive_init)
    dlv_batch = -np.log((nlive_init + nlive_batch + 1.) / (nlive_init + nlive_batch))
    logvol = 0.
    logvols = []
    df_new = df.sort_values('logl').reset_index(drop=True)
    is_batch = df_new.logvol.isna()
    dlvs = np.ones(ntot, dtype=float)*dlv_init
    dlvs[is_batch] = dlv_batch
    logvols = np.cumsum(dlvs)
    df_new.logvol = logvols
    # Compute weights, h, lnz
    h = 0.
    logz = -1e300
    loglstar = -1e300
    logzvar = 0.
    logvols_pad = np.concatenate(([0.], logvols))
    logdvols = logsumexp(a=np.c_[logvols_pad[:-1], logvols_pad[1:]],
                        axis=1, b=np.c_[np.ones(ntot), -np.ones(ntot)])
    logdvols += np.log(0.5)
    dlvs = logvols_pad[:-1] - logvols_pad[1:]
    logwts = []
    logzs = []
    logzvars = []
    hs = []
    for i in range(ntot):
        loglstar_new = df_new.logl[i]
        logdvol, dlv = logdvols[i], dlvs[i]
        logwt = np.logaddexp(loglstar_new, loglstar) + logdvol
        logz_new = np.logaddexp(logz, logwt)
        lzterm = (np.exp(loglstar - logz_new) * loglstar + 
                  np.exp(loglstar_new - logz_new) * loglstar_new)
        h_new = (np.exp(logdvol) * lzterm + 
                np.exp(logz - logz_new) * (h + logz) - logz_new)
        dh = h_new - h
        h = h_new
        logz = logz_new
        logzvar += 2.*dh*dlv
        loglstar = loglstar_new
        logwts.append(logwt)
        logzs.append(logz)
        logzvars.append(logzvar)
        hs.append(h)
    df_new.logwt = np.array(logwts)
    df_new.logz = np.array(logzs)
    df_new.logzerr = np.sqrt(np.array(logzvars))
    df_new.h = np.array(hs)
    return df_new

In [11]:
df_new = merge_batches(df, 300, 100)

In [12]:
df_new.tail()

,niter,nc,eff,logl,logwt,logvol,logz,logzerr,h,delta_logz,time_elapsed,logfeh,logdust,logNpix,tau,dmod
6016,2792.0,9.0,18.129,-0.002834,-22.689915,-16.695519,-5.799515,0.167865,4.067124,NaN,134810.0,-0.229550,-0.80164,3.2180,3.62080,29.650
6017,2793.0,1.0,18.135,-0.002712,-22.692253,-16.698016,-5.799514,0.167865,4.067124,NaN,134810.0,-0.040548,-1.20160,2.9675,2.49680,28.789
6018,3206.0,1.0,19.304,-0.002707,-22.694687,-16.700513,-5.799514,0.167865,4.067124,NaN,145820.0,-0.009662,-1.49050,2.9535,1.86720,28.776
6019,2794.0,5.0,18.142,-0.002489,-22.697072,-16.703010,-5.799514,0.167865,4.067124,NaN,134810.0,0.002810,-1.36760,3.0073,2.72280,28.973
6020,2795.0,5.0,18.148,-0.002318,-22.699375,-16.705506,-5.799514,0.167865,4.067124,NaN,134810.0,-0.259830,-1.17500,3.0260,0.28332,29.078
